In [1]:
#attempt at cleaning up data 

import pandas as pd

df = pd.read_csv('df_may8_730PM.csv')

In [2]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country,Date,ENTRYID,Gender,Nationality,Publisher_Clean,Rank,...,translated_title_embedding,untranslated_title,untranslated_titles,untranslated_titles_embeddings,Best_Match_Title_embeddings,Cosine_Distance_Untranslated,Cosine_Distance_Translated,Selected_Title,Selected_Cosine_Distance,Selected_Source
0,0,2,2,Spain,6/1/18,2018-06Spain8,m,United States,HarperCollins,8.0,...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,NaN,woman in the window,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal
1,1,3,3,Spain,5/1/18,2018-05Spain4,m,United States,HarperCollins,4.0,...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,NaN,woman in the window,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal
2,2,4,4,Spain,4/1/18,2018-04Spain5,m,United States,HarperCollins,5.0,...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,['Woman in the window'],woman in the window,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal
3,3,5,5,United,5/1/18,2018-05United States9,m,United States,HarperCollins,9.0,...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,['Woman in the window'],woman in the window,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal
4,4,6,6,United,2/1/18,2018-02United States1,m,United States,HarperCollins,1.0,...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,['Woman in the window'],woman in the window,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,8985,8986,Egypt,11/1/13,2013-11Egypt10,m,Egypt,Sunrise House,10.0,...,[ 7.53752708e-01 6.75643086e-01 3.44330996e-...,NaN,adajiyu riwayah la ahad yanamu fi aliskandari...,[-3.89930271e-02 7.54797697e-01 4.37511325e-...,[ 7.53752708e-01 6.75643086e-01 3.44330996e-...,0.134122,1.000000,adajiyu riwayah la ahad yanamu fi aliskandari...,1.000000,untranslated
5735,5735,8986,8987,Germany,4/1/14,2014-04Germany8,m; w,Germany; Germany,Droemer Knaur,8.0,...,[-3.80193859e-01 -3.02407265e-01 6.11111104e-...,NaN,fluch des magiers roman list der katzenfrau ro...,[ 1.90752670e-01 7.04852700e-01 1.72644779e-...,[-1.81749128e-02 2.81843275e-01 1.16053414e+...,0.299220,0.523436,fluch des magiers roman list der katzenfrau ro...,0.523436,untranslated
5736,5736,8987,8989,India,6/1/15,2015-06India10,m,India,Westland,10.0,...,[ 0.16614723 0.6165323 1.1202984 0.002846...,NaN,NaN,[ 6.88907504e-03 -5.33142984e-01 2.44534421e+...,[],0.506632,NaN,NaN,0.506632,translated
5737,5737,8989,8993,China,8/1/18,2018-08China3,m,China,Beijing October,3.0,...,[ 6.83612764e-01 2.10637823e-01 3.52317005e-...,NaN,li tai jen wu nien li pei chuan tsung piao 195...,[-1.86778113e-01 4.73393649e-01 9.86138225e-...,[ 4.48107839e-01 -2.67551869e-01 9.00258303e-...,0.169283,0.338207,li tai jen wu nien li pei chuan tsung piao 195...,0.338207,untranslated


In [3]:
#untranslated titles still contains lists, just without commas,
#original lists with commas present, but incorrectly indexed in untranslated_title for reference

#let's start over from title_list maybe, and select the best match?

from fuzzywuzzy import fuzz

def find_best_match(row):
    best_match_score = 0
    best_match_index = -1
    best_match_title = ''
    
    if isinstance(row['title_list'], str):
        title_list = [row['title_list']]
    elif isinstance(row['title_list'], list):
        title_list = row['title_list']
    else:
        title_list = []
    
    for idx, title in enumerate(title_list):
        match_score = fuzz.token_sort_ratio(row['Title'], title)
        if match_score > best_match_score:
            best_match_score = match_score
            best_match_index = idx
            best_match_title = title
    
    row['Best_Match_Index'] = best_match_index
    row['Best_Match_Title_Untranslated'] = best_match_title
    
    return row

# Apply the function to each row
df = df.apply(find_best_match, axis=1)

# Group by author and select the row with the highest match score
best_matches = df.groupby('author').apply(lambda x: x.loc[x['Best_Match_Index'].idxmax()])

# Now best_matches contains only the rows with the best match for each author

/opt/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country,Date,ENTRYID,Gender,Nationality,Publisher_Clean,Rank,...,untranslated_titles,untranslated_titles_embeddings,Best_Match_Title_embeddings,Cosine_Distance_Untranslated,Cosine_Distance_Translated,Selected_Title,Selected_Cosine_Distance,Selected_Source,Best_Match_Index,Best_Match_Title_Untranslated
0,0,2,2,Spain,6/1/18,2018-06Spain8,m,United States,HarperCollins,8.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
1,1,3,3,Spain,5/1/18,2018-05Spain4,m,United States,HarperCollins,4.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
2,2,4,4,Spain,4/1/18,2018-04Spain5,m,United States,HarperCollins,5.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
3,3,5,5,United,5/1/18,2018-05United States9,m,United States,HarperCollins,9.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
4,4,6,6,United,2/1/18,2018-02United States1,m,United States,HarperCollins,1.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,8985,8986,Egypt,11/1/13,2013-11Egypt10,m,Egypt,Sunrise House,10.0,...,"['Adājiyū : riwāyah', 'Lā aḥad yanāmu fi...",[-3.89930271e-02 7.54797697e-01 4.37511325e-...,[ 7.53752708e-01 6.75643086e-01 3.44330996e-...,0.134122,1.000000,adajiyu riwayah la ahad yanamu fi aliskandari...,1.000000,untranslated,0,No one sleeps in Alexandria
5735,5735,8986,8987,Germany,4/1/14,2014-04Germany8,m; w,Germany; Germany,Droemer Knaur,8.0,...,"['Fluch des Magiers Roman', 'List der Katzenfr...",[ 1.90752670e-01 7.04852700e-01 1.72644779e-...,[-1.81749128e-02 2.81843275e-01 1.16053414e+...,0.299220,0.523436,fluch des magiers roman list der katzenfrau ro...,0.523436,untranslated,0,Stern der Göttin Roman
5736,5736,8987,8989,India,6/1/15,2015-06India10,m,India,Westland,10.0,...,,[ 6.88907504e-03 -5.33142984e-01 2.44534421e+...,[],0.506632,NaN,NaN,0.506632,translated,-1,
5737,5737,8989,8993,China,8/1/18,2018-08China3,m,China,Beijing October,3.0,...,"['Li tai jen wu nien li pei chuan tsung piao, ...",[-1.86778113e-01 4.73393649e-01 9.86138225e-...,[ 4.48107839e-01 -2.67551869e-01 9.00258303e-...,0.169283,0.338207,li tai jen wu nien li pei chuan tsung piao 195...,0.338207,untranslated,0,Qie wen zhai ji : 16 juan


In [7]:
# from fuzzywuzzy import fuzz
# import ast  # Library for parsing strings into lists

# def find_best_match(row):
#     best_match_score = 0
#     best_match_title = ''
    
#     # Parse the string representation of list if present
#     if isinstance(row['Best_Match_Title_Untranslated'], str):
#         title_list = ast.literal_eval(row['Best_Match_Title_Untranslated'])  # Convert string to list
#     elif isinstance(row['Best_Match_Title_Untranslated'], list):
#         title_list = row['Best_Match_Title_Untranslated']    
#     for idx, title in enumerate(title_list):
#         match_score = fuzz.token_sort_ratio(row['Title'], title)
#         if match_score > best_match_score:
#             best_match_score = match_score
#             best_match_title = title
    
#     row['Best_Match_Title_Untranslated'] = best_match_title
    
#     return row

# # Apply the function to each row
# df = df.apply(find_best_match, axis=1)

# # No need for further grouping, as each row will now have the best match title


SyntaxError: invalid syntax (<unknown>, line 0)

In [8]:
df['untranslated_titles'] = df['Best_Match_Title_Untranslated'] 

In [9]:
from fuzzywuzzy import fuzz

def find_best_match(row):
    best_match_score = 0
    best_match_title = ''
    
    # Parse the string representation of list if present
    if isinstance(row['untranslated_titles'], str):
        try:
            title_list = ast.literal_eval(row['untranslated_titles'])  # Convert string to list
        except (ValueError, SyntaxError):
            title_list = []  # In case of invalid syntax, treat it as an empty list
    elif isinstance(row['untranslated_titles'], list):
        title_list = row['untranslated_titles']
    else:
        title_list = []
    
    for idx, title in enumerate(title_list):
        match_score = fuzz.token_sort_ratio(row['Title'], title)
        if match_score > best_match_score:
            best_match_score = match_score
            best_match_title = title
    
    row['Best_Match_Title_Untranslated'] = best_match_title
    
    return row

# Apply the function to each row
df = df.apply(find_best_match, axis=1)


In [10]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country,Date,ENTRYID,Gender,Nationality,Publisher_Clean,Rank,...,untranslated_titles,untranslated_titles_embeddings,Best_Match_Title_embeddings,Cosine_Distance_Untranslated,Cosine_Distance_Translated,Selected_Title,Selected_Cosine_Distance,Selected_Source,Best_Match_Index,Best_Match_Title_Untranslated
0,0,2,2,Spain,6/1/18,2018-06Spain8,m,United States,HarperCollins,8.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
1,1,3,3,Spain,5/1/18,2018-05Spain4,m,United States,HarperCollins,4.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
2,2,4,4,Spain,4/1/18,2018-04Spain5,m,United States,HarperCollins,5.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
3,3,5,5,United,5/1/18,2018-05United States9,m,United States,HarperCollins,9.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
4,4,6,6,United,2/1/18,2018-02United States1,m,United States,HarperCollins,1.0,...,['Woman in the window'],[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,[ 9.03542995e-01 -5.74119329e-01 4.56858873e-...,0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,8985,8986,Egypt,11/1/13,2013-11Egypt10,m,Egypt,Sunrise House,10.0,...,"['Adājiyū : riwāyah', 'Lā aḥad yanāmu fi...",[-3.89930271e-02 7.54797697e-01 4.37511325e-...,[ 7.53752708e-01 6.75643086e-01 3.44330996e-...,0.134122,1.000000,adajiyu riwayah la ahad yanamu fi aliskandari...,1.000000,untranslated,0,No one sleeps in Alexandria
5735,5735,8986,8987,Germany,4/1/14,2014-04Germany8,m; w,Germany; Germany,Droemer Knaur,8.0,...,"['Fluch des Magiers Roman', 'List der Katzenfr...",[ 1.90752670e-01 7.04852700e-01 1.72644779e-...,[-1.81749128e-02 2.81843275e-01 1.16053414e+...,0.299220,0.523436,fluch des magiers roman list der katzenfrau ro...,0.523436,untranslated,0,Stern der Göttin Roman
5736,5736,8987,8989,India,6/1/15,2015-06India10,m,India,Westland,10.0,...,,[ 6.88907504e-03 -5.33142984e-01 2.44534421e+...,[],0.506632,NaN,NaN,0.506632,translated,-1,
5737,5737,8989,8993,China,8/1/18,2018-08China3,m,China,Beijing October,3.0,...,"['Li tai jen wu nien li pei chuan tsung piao, ...",[-1.86778113e-01 4.73393649e-01 9.86138225e-...,[ 4.48107839e-01 -2.67551869e-01 9.00258303e-...,0.169283,0.338207,li tai jen wu nien li pei chuan tsung piao 195...,0.338207,untranslated,0,Qie wen zhai ji : 16 juan


In [ ]:
#note for after I eat dinner... return here and change column names below so can run these over the dataframe now

In [12]:
df['untranslated_titles'] = df['Best_Match_Title_Untranslated']

In [14]:
df['Best_Match_Title'].head(50)

0                                   woman in the window
1                                   woman in the window
2                                   woman in the window
3                                   woman in the window
4                                   woman in the window
5                                   woman in the window
6                                   woman in the window
7                                   woman in the window
8                                   woman in the window
9                        black rock white city  a novel
10                                          friend zone
11                                                  NaN
12                                        rough weather
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [18]:
import re
processed_df = df

def clean_title(title):
    if title is not None:
        # Convert to lowercase
        title = title.lower()
        # Remove punctuation, quotation marks, and other special characters
        title = re.sub(r'[^\w\s]', '', title)
    return title

processed_df['Title'] = processed_df['Title'].apply(clean_title)
processed_df['untranslated_titles'] = processed_df['untranslated_titles'].apply(clean_title)
# processed_df['Best_Match_Title'] = processed_df['Best_Match_Title'].apply(clean_title)

In [20]:
#set up the SentenceBERT model
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load Sentence-BERT model
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [25]:
df = processed_df

def create_embeddings(titles):
    if titles is not None:
        embeddings = model.encode(titles)
        return embeddings
    else:
        return []


# Apply the function to create embeddings for extracted titles
df['Title_Embeddings'] = df['Title'].apply(create_embeddings)
df['untranslated_titles_embeddings'] = df['untranslated_titles'].apply(create_embeddings)
# df['Best_Match_Title_embeddings'] = processed_df['Best_Match_Title'].apply(create_embeddings)
# df['untranslated_title_embedding'] = df['Best_Title_Match_Orig_Untrans'].apply(create_embeddings)

# Display the DataFrame with title embeddings
# print(df)

In [24]:
df['Best_Match_Title'] = df['Best_Match_Title'].astype(str)
df['Best_Match_Title_embeddings'] = df['Best_Match_Title'].apply(create_embeddings)


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine distances between original and best match titles
df['Cosine_Distance_Untranslated'] = df.apply(lambda row: cosine_similarity([row['Title_Embeddings']], [row['untranslated_titles_embeddings']])[0][0] if len(row['untranslated_titles_embeddings']) > 0 else None, axis=1)

#repeat for translated titles here:
df['Cosine_Distance_Translated'] = df.apply(lambda row: cosine_similarity([row['Title_Embeddings']], [row['Best_Match_Title_embeddings']])[0][0] if len(row['Best_Match_Title_embeddings']) > 0 else None, axis=1)


In [27]:
# Function to select the title and cosine distance based on the highest cosine distance
def select_title_and_distance(row):
    cosine_original_best_match = row['Cosine_Distance_Translated']
    cosine_original_translated = row['Cosine_Distance_Untranslated']
    
    if cosine_original_best_match > cosine_original_translated:
        selected_title = row['untranslated_titles']
        selected_cosine_distance = cosine_original_best_match
        selected_source = 'untranslated'
    else:
        selected_title = row['Best_Match_Title']
        selected_cosine_distance = cosine_original_translated
        selected_source = 'translated'
    
    return selected_title, selected_cosine_distance, selected_source

# Apply the function to each row and create new columns
df['Selected_Title'], df['Selected_Cosine_Distance'], df['Selected_Source'] = zip(*df.apply(select_title_and_distance, axis=1))


In [28]:
#add condition for equality...

import numpy as np

# Check if cosine distances are equal
df['Selected_Source'] = np.where(df['Cosine_Distance_Translated'] == df['Cosine_Distance_Untranslated'], 'equal', df['Selected_Source'])



In [29]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country,Date,ENTRYID,Gender,Nationality,Publisher_Clean,Rank,...,untranslated_titles,untranslated_titles_embeddings,Best_Match_Title_embeddings,Cosine_Distance_Untranslated,Cosine_Distance_Translated,Selected_Title,Selected_Cosine_Distance,Selected_Source,Best_Match_Index,Best_Match_Title_Untranslated
0,0,2,2,Spain,6/1/18,2018-06Spain8,m,United States,HarperCollins,8.0,...,woman in the window,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
1,1,3,3,Spain,5/1/18,2018-05Spain4,m,United States,HarperCollins,4.0,...,woman in the window,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
2,2,4,4,Spain,4/1/18,2018-04Spain5,m,United States,HarperCollins,5.0,...,woman in the window,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
3,3,5,5,United,5/1/18,2018-05United States9,m,United States,HarperCollins,9.0,...,woman in the window,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
4,4,6,6,United,2/1/18,2018-02United States1,m,United States,HarperCollins,1.0,...,woman in the window,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,8985,8986,Egypt,11/1/13,2013-11Egypt10,m,Egypt,Sunrise House,10.0,...,no one sleeps in alexandria,"[0.7537527, 0.6756431, 0.344331, -0.05734726, ...","[0.7537527, 0.6756431, 0.344331, -0.05734726, ...",1.000000,1.000000,no one sleeps in alexandria,1.000000,equal,0,No one sleeps in Alexandria
5735,5735,8986,8987,Germany,4/1/14,2014-04Germany8,m; w,Germany; Germany,Droemer Knaur,8.0,...,stern der gottin roman,"[-0.018174913, 0.28184327, 1.1605341, 0.490151...","[-0.018174913, 0.28184327, 1.1605341, 0.490151...",0.523436,0.523436,stern der gottin roman,0.523436,equal,0,Stern der Göttin Roman
5736,5736,8987,8989,India,6/1/15,2015-06India10,m,India,Westland,10.0,...,,"[0.006889075, -0.533143, 2.4453442, 0.25165877...","[0.029933482, -0.156202, 1.251409, -0.29579616...",0.506632,0.462815,nan,0.506632,translated,-1,
5737,5737,8989,8993,China,8/1/18,2018-08China3,m,China,Beijing October,3.0,...,qie wen zhai ji 16 juan,"[-0.16479598, 0.6650823, 1.0153482, -0.1090161...","[0.44810784, -0.26755187, 0.9002583, -0.040235...",0.251762,0.338207,qie wen zhai ji 16 juan,0.338207,untranslated,0,Qie wen zhai ji : 16 juan


In [30]:
df.to_csv('df_may8_8PM.csv')

In [33]:
#have to repeat translation tho...
# Initialize EasyNMT for translation
from easynmt import EasyNMT

model_name = 'mbart50_m2m'
# Create the translator
translator = EasyNMT(model_name=model_name, language_detection = 'fasttext')

In [31]:
def translate_title(title):
    if title is not None:  # Check if title is not None
        translated_title = translator.translate(title, target_lang='en')
        return translated_title
    else:
        return 'None'  # Return 'None' as a string if title is None


In [38]:
# def retranslate(row):
#     index = row.name
# #     print("Processing index:", index)
#     # Print "next chunk!" every 200 rows
#     if index % 200 == 0 and index != 0:
#         print("next chunk!")
    
#     if index % 50 == 0:
#         print("Processing next 50 rows...")
    

#     if row['Selected_Source'] in ['equal', 'untranslated']:
#         # Add your translation logic here
#         # For example:
#         translated_title = translate_title(row['Best_Match_Title_Untranslated'])
#         return translated_title
#     else:
#         return row['Title']  # If not equal or untranslated, return the original title

# # Apply the function to each row
# df['Retranslated_Title'] = df.apply(retranslate, axis=1)


Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
next chunk!
Processing next 50 rows...


KeyboardInterrupt: 

In [41]:
def retranslate(row):
    index = row.name  # Get the index of the row
    
    if index % 200 == 0 and index != 0:
        print("Start index:", index - 199)
        print("End index:", index)
        print("next chunk!")
    
    if index % 50 == 0:
        print("Processing next 50 rows...")
    
    if row['Selected_Source'] in ['equal', 'untranslated']:
        # Add your translation logic here
        # For example:
        translated_title = translate_title(row['Best_Match_Title_Untranslated'])
        return translated_title
    else:
        return row['Best_Match_Title_Untranslated']  # If not equal or untranslated, return the original title

# Apply the function to each row
df['Retranslated_Title'] = df.apply(retranslate, axis=1)


Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Start index: 1
End index: 200
next chunk!
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Start index: 201
End index: 400
next chunk!
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Start index: 401
End index: 600
next chunk!
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Start index: 601
End index: 800
next chunk!
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Start index: 801
End index: 1000
next chunk!
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Start index: 1001
End index: 1200
next chunk!
Processing next 50 rows...
Processing next 50 rows...
Processing next 50 rows...
Proces

In [42]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country,Date,ENTRYID,Gender,Nationality,Publisher_Clean,Rank,...,untranslated_titles_embeddings,Best_Match_Title_embeddings,Cosine_Distance_Untranslated,Cosine_Distance_Translated,Selected_Title,Selected_Cosine_Distance,Selected_Source,Best_Match_Index,Best_Match_Title_Untranslated,Retranslated_Title
0,0,2,2,Spain,6/1/18,2018-06Spain8,m,United States,HarperCollins,8.0,...,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window
1,1,3,3,Spain,5/1/18,2018-05Spain4,m,United States,HarperCollins,4.0,...,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window
2,2,4,4,Spain,4/1/18,2018-04Spain5,m,United States,HarperCollins,5.0,...,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window
3,3,5,5,United,5/1/18,2018-05United States9,m,United States,HarperCollins,9.0,...,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window
4,4,6,6,United,2/1/18,2018-02United States1,m,United States,HarperCollins,1.0,...,"[0.903543, -0.5741193, 0.45685887, 0.032110155...","[0.903543, -0.5741193, 0.45685887, 0.032110155...",0.987460,0.987460,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,8985,8986,Egypt,11/1/13,2013-11Egypt10,m,Egypt,Sunrise House,10.0,...,"[0.7537527, 0.6756431, 0.344331, -0.05734726, ...","[0.7537527, 0.6756431, 0.344331, -0.05734726, ...",1.000000,1.000000,no one sleeps in alexandria,1.000000,equal,0,No one sleeps in Alexandria,No one sleeps in Alexandria
5735,5735,8986,8987,Germany,4/1/14,2014-04Germany8,m; w,Germany; Germany,Droemer Knaur,8.0,...,"[-0.018174913, 0.28184327, 1.1605341, 0.490151...","[-0.018174913, 0.28184327, 1.1605341, 0.490151...",0.523436,0.523436,stern der gottin roman,0.523436,equal,0,Stern der Göttin Roman,Star of the Goddess novel
5736,5736,8987,8989,India,6/1/15,2015-06India10,m,India,Westland,10.0,...,"[0.006889075, -0.533143, 2.4453442, 0.25165877...","[0.029933482, -0.156202, 1.251409, -0.29579616...",0.506632,0.462815,nan,0.506632,translated,-1,,
5737,5737,8989,8993,China,8/1/18,2018-08China3,m,China,Beijing October,3.0,...,"[-0.16479598, 0.6650823, 1.0153482, -0.1090161...","[0.44810784, -0.26755187, 0.9002583, -0.040235...",0.251762,0.338207,qie wen zhai ji 16 juan,0.338207,untranslated,0,Qie wen zhai ji : 16 juan,Qie wen zhai ji: 16 juan


In [ ]:
#overlap and fuzzy metrics aren't in this notebook yet, add them back...


In [44]:
from fuzzywuzzy import fuzz

# Calculate fuzzywuzzy ratio between Title and best_match
df['FuzzyWuzzy_Ratio_Title_BestMatch'] = df.apply(lambda row: fuzz.ratio(row['Title'], row['Best_Match_Title_Untranslated']), axis=1)

# Calculate fuzzywuzzy ratio between Title and Translated_Title2
df['FuzzyWuzzy_Ratio_Title_Translated2'] = df.apply(lambda row: fuzz.ratio(row['Title'], row['Best_Match_Title']), axis=1)


In [45]:
def check_overlap_without_stopwords(text1, text2):
    tokens1 = set(tokenize(text1))
    tokens2 = set(tokenize(text2))  # Removed [0] here
    overlap = tokens1.intersection(tokens2)  # Fixed here as well
    return len(overlap) > 3


In [48]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Get English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and tokenize text
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/beccacohen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/beccacohen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
# Check for direct overlap between Title and best_match after removing stopwords
df['Direct_Overlap_Title_BestMatch'] = df.apply(lambda row: check_overlap_without_stopwords(row['Title'], row['Best_Match_Title_Untranslated']), axis=1)
df['Direct_Overlap_Title_Translated'] = df.apply(lambda row: check_overlap_without_stopwords(row['Title'], row['Best_Match_Title']), axis=1)


In [50]:
df.to_csv('df_may8_1036.csv')

In [51]:
from fuzzywuzzy import process

def fill_empty_titles(row):
    if pd.isnull(row['untranslated_titles']):
        # Get the title list
        title_list = row['title_list']
        
        # Check if title_list is a string representation of a list
        if isinstance(title_list, str):
            # Split the string by commas and extract titles
            title_list = [t.strip() for t in title_list.split(',')]
        
        # If title_list is not a list, treat it as an empty list
        if not isinstance(title_list, list):
            title_list = []
        
        # Check if title_list is not empty
        if title_list:
            # Find the best match from title_list for the blank title
            best_match = process.extractOne(row['Title'], title_list)[0]
            return best_match
        else:
            # If title_list is empty, return None
            return None
    else:
        # If untranslated_titles is not blank, return the original value
        return row['untranslated_titles']

# Apply the function to each row
df['untranslated_titles'] = df.apply(fill_empty_titles, axis=1)


In [53]:
df['untranslated_titles']

0               woman in the window
1               woman in the window
2               woman in the window
3               woman in the window
4               woman in the window
                   ...             
5734    no one sleeps in alexandria
5735         stern der gottin roman
5736                               
5737       qie wen zhai ji  16 juan
5738                               
Name: untranslated_titles, Length: 5739, dtype: object

In [55]:
# Check which cells in the 'untranslated_titles' column are null
blank_cells = df[df['untranslated_titles'].isnull()]

# Display the rows where 'untranslated_titles' is null


In [93]:
import re

import fasttext

# Load the language detection model
print("Loading language detection model...")
language_model = fasttext.load_model('lid.176.bin')
print("Language detection model loaded successfully!")

def detect_language(text):
    # Make predictions using the language detection model
    predictions = language_model.predict(text, k=1)
    
    # Print predictions for debugging
    print("Predictions:", predictions)
    
    # Extract the predicted language label
    language_label = str(predictions[0])
    print("Language label:", language_label)
    
    # Extract the language code using regex
    language_code_match = re.search(r"__label__(\w+)", language_label)
    if language_code_match:
        language_code = language_code_match.group(1)
        return language_code
    else:
        print("Warning: Invalid language label!")
        return None


Loading language detection model...
Language detection model loaded successfully!


In [92]:
# Test the language detection function
text = "This is a test sentence."
print("Detecting language for text:", text)
language_code = detect_language(text)
print("Detected language code:", language_code)

Detecting language for text: This is a test sentence.
Predictions: (('__label__en',), array([0.97088009]))
Language label: ('__label__en',)
Detected language code: en


In [94]:
# Function to detect language for each text in the DataFrame
def detect_language_for_row(row):
    untranslated_title = row['untranslated_titles']
    language_code = detect_language(untranslated_title)
    return language_code

# Apply the function to each row in the DataFrame
df['detected_language'] = df.apply(detect_language_for_row, axis=1)


Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.85653824]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8814773]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8141315]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.1104493]))
Lang

In [95]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country,Date,ENTRYID,Gender,Nationality,Publisher_Clean,Rank,...,Selected_Title,Selected_Cosine_Distance,Selected_Source,Best_Match_Index,Best_Match_Title_Untranslated,Retranslated_Title,FuzzyWuzzy_Ratio_Title_Translated2,Direct_Overlap_Title_BestMatch,Direct_Overlap_Title_Translated,detected_language
0,0,2,2,Spain,6/1/18,2018-06Spain8,m,United States,HarperCollins,8.0,...,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window,90,False,True,en
1,1,3,3,Spain,5/1/18,2018-05Spain4,m,United States,HarperCollins,4.0,...,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window,90,False,True,en
2,2,4,4,Spain,4/1/18,2018-04Spain5,m,United States,HarperCollins,5.0,...,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window,90,False,True,en
3,3,5,5,United,5/1/18,2018-05United States9,m,United States,HarperCollins,9.0,...,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window,90,False,True,en
4,4,6,6,United,2/1/18,2018-02United States1,m,United States,HarperCollins,1.0,...,woman in the window,0.987460,equal,0,Woman in the window,Woman in the window,90,False,True,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,8985,8986,Egypt,11/1/13,2013-11Egypt10,m,Egypt,Sunrise House,10.0,...,no one sleeps in alexandria,1.000000,equal,0,No one sleeps in Alexandria,No one sleeps in Alexandria,100,False,True,en
5735,5735,8986,8987,Germany,4/1/14,2014-04Germany8,m; w,Germany; Germany,Droemer Knaur,8.0,...,stern der gottin roman,0.523436,equal,0,Stern der Göttin Roman,Star of the Goddess novel,28,False,False,de
5736,5736,8987,8989,India,6/1/15,2015-06India10,m,India,Westland,10.0,...,nan,0.506632,translated,-1,,,7,False,False,en
5737,5737,8989,8993,China,8/1/18,2018-08China3,m,China,Beijing October,3.0,...,qie wen zhai ji 16 juan,0.338207,untranslated,0,Qie wen zhai ji : 16 juan,Qie wen zhai ji: 16 juan,20,False,False,en


In [97]:
# Function to detect language for each text in the DataFrame
def detect_language_for_row(row):
    untranslated_title = row['Retranslated_Title']
    language_code = detect_language(untranslated_title)
    return language_code

# Apply the function to each row in the DataFrame
df['Retranslation_Language_progress'] = df.apply(detect_language_for_row, axis=1)


Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.8405202]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.80943429]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.44040433]))
Language label: ('__label__en',)
Predictions: (('__label__en',), array([0.1104493]))
Language la

In [100]:
df.to_csv('language_detect_version.csv')

In [ ]:
def retranslate_if_needed(row):
    if row['Retranslation_Language_progress'] != 'en':
        translated_title = translate_title(row['Retranslated_Title'])
        return translated_title
    else:
        return row['Retranslated_Title']

# Apply the function to each row in the DataFrame
df['Translated_Titles_Retranslated'] = df.apply(retranslate_if_needed, axis=1)


In [ ]:
def retranslate_if_needed(row):
    global processed_rows, start_idx, end_idx
    processed_rows += 1
    
    if processed_rows % 50 == 0:
        print("Processed", processed_rows, "rows...")
    
    if processed_rows % 200 == 0:
        print("New chunk!")
    
    if processed_rows % CHUNK_SIZE == 0:
        print("Start index:", start_idx, "End index:", end_idx)
    
    if row['Retranslation_Language_progress'] != 'en':
        translated_title = translate_title(row['Retranslated_Title'])
        return translated_title
    else:
        return row['Retranslated_Title']


# Apply the function to each row in the DataFrame
processed_rows = 0
CHUNK_SIZE = 200  # Define chunk size
for start_idx in range(0, len(df), CHUNK_SIZE):
    end_idx = min(start_idx + CHUNK_SIZE, len(df))
    chunk = df.iloc[start_idx:end_idx]
    chunk['New_Retranslated_Titles'] = chunk.apply(retranslate_if_needed, axis=1)
    df.iloc[start_idx:end_idx] = chunk


Processed 50 rows...
Processed 100 rows...
Processed 150 rows...
Processed 200 rows...
New chunk!
Start index: 0 End index: 200


/var/folders/ct/0m2g6vn91w117r0qm06rm0vm0000gn/T/ipykernel_94979/2233679017.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['New_Retranslated_Titles'] = chunk.apply(retranslate_if_needed, axis=1)


Processed 250 rows...
Processed 300 rows...
Processed 350 rows...
Processed 400 rows...
New chunk!
Start index: 200 End index: 400


/var/folders/ct/0m2g6vn91w117r0qm06rm0vm0000gn/T/ipykernel_94979/2233679017.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['New_Retranslated_Titles'] = chunk.apply(retranslate_if_needed, axis=1)


Processed 450 rows...
Processed 500 rows...
Processed 550 rows...
Processed 600 rows...
New chunk!
Start index: 400 End index: 600


...

/var/folders/ct/0m2g6vn91w117r0qm06rm0vm0000gn/T/ipykernel_94979/2233679017.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['New_Retranslated_Titles'] = chunk.apply(retranslate_if_needed, axis=1)


Processed 1550 rows...
Processed 1600 rows...
New chunk!
Start index: 1400 End index: 1600
Processed 1650 rows...
Processed 1700 rows...


/var/folders/ct/0m2g6vn91w117r0qm06rm0vm0000gn/T/ipykernel_94979/2233679017.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['New_Retranslated_Titles'] = chunk.apply(retranslate_if_needed, axis=1)


Processed 1750 rows...
Processed 1800 rows...
New chunk!
Start index: 1600 End index: 1800


/var/folders/ct/0m2g6vn91w117r0qm06rm0vm0000gn/T/ipykernel_94979/2233679017.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['New_Retranslated_Titles'] = chunk.apply(retranslate_if_needed, axis=1)
